# Football matches result prediction

Let's try to predict *Serie A* matches result (i.e. home win, away win or draw) with a RNN.

## Introduction

- The dataset was created by scraping *Serie A* matches data starting from season 2005-06 to season 2020-21
- Cup matches (*Champions League*, *Europa League*, *Coppa Italia*) played over the course of each season were not taken into account

In [2827]:
import re

import pandas as pd
import torch
import torch.nn as nn
import numpy as np
from sklearn.preprocessing import LabelBinarizer
from torch import optim
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from train_utils import AverageMeter

from MatchResult import MatchResult

In [2828]:
match_cols = ['season', 'round'] + \
             ['date', 'time', 'referee', 'home_team', 'away_team', 'home_team_score', 'away_team_score'] + \
             ['home_team_coach'] + \
             ['home_player_' + str(i) for i in range(1, 12)] + \
             ['home_substitute_' + str(i) for i in range(1, 8)] + \
             ['away_team_coach'] + \
             ['away_player_' + str(i) for i in range(1, 12)] + \
             ['away_substitute_' + str(i) for i in range(1, 8)]

In [2829]:
train_raw = pd.read_csv('train-raw.csv')
train_raw.head()

,season,round,date,time,referee,home_team,away_team,home_team_score,away_team_score,home_team_coach,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,28/08/2005,15:00,MASSIMO DE,ASCOLI,MILAN,1,1,Massimo Silva,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
1,2005-06,1,27/08/2005,20:30,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,2,1,Cesare Prandelli,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
2,2005-06,1,28/08/2005,15:00,TIZIANO PIERI,PARMA,PALERMO,1,1,Mario Beretta,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
3,2005-06,1,28/08/2005,15:00,PAOLO TAGLIAVENTO,INTER,TREVISO,3,0,Roberto Mancini,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto
4,2005-06,1,27/08/2005,18:00,GIANLUCA ROCCHI,LIVORNO,LECCE,2,1,Roberto Donadoni,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino


In [2830]:
# test_raw = pd.read_csv('test-raw.csv')
# test_raw.head()

## Data visualization

Let's inspect our data a little bit more

In [2831]:
# todo

## Dataset pre-processing
Now let's clean our raw data and add some additional features.

In [2832]:
df = pd.DataFrame(train_raw)
df = df[:200]

In [2833]:
# convert date str to datetime
df['date'] = pd.to_datetime(df['date'], infer_datetime_format=True)
# sort by date column
df = df.sort_values(by='date')
df = df.reset_index(drop=True)
# round values to int
df['round'] = df['round'].astype(int)

In [2834]:
# utility methods
def get_team_and_historical_index_from_match_team_id(match_team_id: str) -> (str, str):
    match_team_name = re.findall("\s+", match_team_id)[0]
    match_team_index = re.findall("\d+", match_team_id)[0]
    return match_team_name, match_team_index


def get_match_by_team_and_round(df: pd.DataFrame, team: str, round: int) -> pd.DataFrame:
    return df[((df['home_team'] == team) | (df['away_team'] == team)) & (df['round'] == round)]


def get_last_n_matches_played_by_team_before_round(df: pd.DataFrame, team: str, round: int, n: int) -> pd.DataFrame:
    last_n_matches = pd.DataFrame()
    for i in range(1, n+1):
        if round - i > 0:
            last_n_matches = pd.concat([last_n_matches, get_match_by_team_and_round(df, team, round - i)])
    return last_n_matches

In [2835]:
a = get_match_by_team_and_round(df, 'ROBUR SIENA', 8)
a

,season,round,date,time,referee,home_team,away_team,home_team_score,away_team_score,home_team_coach,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
77,2005-06,8,2005-10-23,15:00,OSCAR GIRARDI,SAMPDORIA,ROBUR SIENA,3,3,Walter Alfredo Novellino,...,Erjon Bogdani,Thomas Locatelli,Enrico Chiesa,Alberto,Jonathan Bachini,Francesco Colonnese,Marco Fortin,Daniele Gastaldello,Matteo Paro,Roberto Nanni


### Additional features

#### Result column
Our model will try to predict match results, i.e. **home win**, **away win** or **draw**, so we need a result column to be used as our target.

In [2836]:
def get_match_result_from_score(home_team_score: int, away_team_score: int) -> MatchResult:
    if home_team_score == away_team_score:
        return MatchResult.draw
    if home_team_score > away_team_score:
        return MatchResult.home
    return MatchResult.away


def add_target_column(df: pd.DataFrame) -> pd.DataFrame:
    results = {'result': []}
    for index, row in df.iterrows():
        results['result'] += [get_match_result_from_score(row['home_team_score'], row['away_team_score']).name]
    df.insert(loc=df.columns.get_loc('home_team_score'), column='result', value=results['result'])
    return df

In [2837]:
# add target column
add_target_column(df)

,season,round,date,time,referee,home_team,away_team,result,home_team_score,away_team_score,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,2005-08-27,20:30,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,home,2,1,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
1,2005-06,1,2005-08-27,18:00,GIANLUCA ROCCHI,LIVORNO,LECCE,home,2,1,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino
2,2005-06,1,2005-08-28,15:00,MASSIMO DE,ASCOLI,MILAN,draw,1,1,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
3,2005-06,1,2005-08-28,15:00,TIZIANO PIERI,PARMA,PALERMO,draw,1,1,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
4,2005-06,1,2005-08-28,15:00,PAOLO TAGLIAVENTO,INTER,TREVISO,home,3,0,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2005-06,20,2006-01-18,20:30,PAOLO TAGLIAVENTO,ROMA,REGGINA,home,3,1,...,Francesco Modesto,Francesco Cozza,Luca Vigiani,Maurizio Lauro,Nicola Amoruso,Simone Missiroli,Ivan Pelizzoli,Davide Biondini,Filippo Carobbio,Simone Cavalli
196,2005-06,20,2006-01-18,20:30,GIANLUCA ROCCHI,MILAN,ASCOLI,home,1,0,...,Cristiano Del Grosso,Sasa Bjelanovic,Fabio Quagliarella,Massimo Paci,Michele Fini,Pasquale Foggia,Carlo Zotti,Riccardo Corallo,Davide Oresti,Marco Ferrante
197,2005-06,20,2006-01-18,20:30,PASQUALE RODOMONTI,LECCE,LIVORNO,draw,0,0,...,Francesco Coco,Ibrahima Bakayoko,Cristiano Lucarelli,Marc Pfertzel,Cesar Prates,Raffaele Palladino,Paolo Acerbis,Stefano Fanucci,Giuseppe Colucci,Paulinho
198,2005-06,20,2006-01-18,20:30,ANDREA ROMEO,CAGLIARI,ROBUR SIENA,home,1,0,...,Cristian Molinaro,Erjon Bogdani,Enrico Chiesa,Rej Volpato,Paolo Negro,Nicola Legrottaglie,Marco Fortin,Francesco Colonnese,Roberto Nanni,NaN


### Rest days features
Rest days are very important for recovery.

In [2838]:
def count_days_between_dates(date1, date2) -> int:
    return (date1 - date2).dt.days

In [2839]:
# for i in range(5):
#     for home_or_away in HomeOrAway:
#         if i == 0:
#             df[f'{home_or_away.name}_team_rest_days'] = count_days_between_dates(df['date'], df[f'{home_or_away.name}_team_history_{i+1}_date'])
#         else:
#             df[f'{home_or_away.name}_team_history_{i}_rest_days'] = count_days_between_dates(df[f'{home_or_away.name}_team_history_{i}_date'], df[f'{home_or_away.name}_team_history_{i+1}_date'])

# todo: cannot count rest days for historical 5th games because we still miss the data about the 6th historical match

In [2840]:
# delete columns referring to the historical 6th matches
# df = df.loc[:, ~df.columns.str.contains('history_6')]

#### Datetime features
Add **year**, **month** and **day** features for all **date** value

In [2841]:
def get_exploded_datetime_values(df: pd.DataFrame) -> dict:
    data = {'year': [], 'month': [], 'day': [], 'hour': []}
    df['time'] = pd.to_datetime(df['time'], format="%H:%M")
    data['year'] += df['date'].map(lambda val: val.year).tolist()
    data['month'] += df['date'].map(lambda val: val.month).tolist()
    data['day'] += df['date'].map(lambda val: val.day).tolist()
    data['hour'] += df['time'].map(lambda val: val.hour).tolist()
    return data


def insert_exploded_datetime_values(df, exploded):
    df.insert(loc=df.columns.get_loc('time'), column='year', value=exploded['year'])
    df.insert(loc=df.columns.get_loc('time'), column='month', value=exploded['month'])
    df.insert(loc=df.columns.get_loc('time'), column='day', value=exploded['day'])
    df.insert(loc=df.columns.get_loc('time'), column='hour', value=exploded['hour'])
    return df


def explode_datetime_values(df: pd.DataFrame) -> pd.DataFrame:
    exploded = get_exploded_datetime_values(df)
    return insert_exploded_datetime_values(df, exploded)


def get_column_names_containing_str(df: pd.DataFrame, substring: str) -> list[str]:
    return df.loc[:,df.columns.str.contains(substring)].columns.values.tolist()

In [2842]:
# explode datetime values
df = explode_datetime_values(df)
# drop date columns
date_cols = get_column_names_containing_str(df, 'date')
df.drop(date_cols, axis=1, inplace=True)
df.drop(['time'], axis=1, inplace=True)

The result of the pre-processing looks like this:

In [2843]:
df.head()

,season,round,year,month,day,hour,referee,home_team,away_team,result,...,away_player_9,away_player_10,away_player_11,away_substitute_1,away_substitute_2,away_substitute_3,away_substitute_4,away_substitute_5,away_substitute_6,away_substitute_7
0,2005-06,1,2005,8,27,20,GIANLUCA PAPARESTA,FIORENTINA,SAMPDORIA,home,...,Lamberto Zauli,Francesco Flachi,Emiliano Bonazzoli,Marco Pisano,Vitaliy Kutuzov,Marco Borriello,Luca Castellazzi,Marco Zamboni,Simone Pavan,Gionata Mingozzi
1,2005-06,1,2005,8,27,18,GIANLUCA ROCCHI,LIVORNO,LECCE,home,...,Alex Pinardi,Aleksei Eremenko,Graziano Pelle,Alfonso Camorani,Jaime Valdes,Giuseppe Cozzolino,Francesco Benussi,Marco Pecorari,Giuseppe Abruzzese,Davide Giorgino
2,2005-06,1,2005,8,28,15,MASSIMO DE,ASCOLI,MILAN,draw,...,Kaka,Andriy Shevchenko,Alberto Gilardino,Marek Jankulovski,Clarence Seedorf,Zeljko Kalac,Gennaro Gattuso,Manuel Rui Costa,Johann Vogel,Dario Simic
3,2005-06,1,2005,8,28,15,TIZIANO PIERI,PARMA,PALERMO,draw,...,Massimo Bonanni,Andrea Caracciolo,Stephen Makinwa,Nicola Santoni,Franco Brienza,Massimo Mutarelli,Giuseppe Biava,Michele Ferri,Mariano Gonzalez,Simone Pepe
4,2005-06,1,2005,8,28,15,PAOLO TAGLIAVENTO,INTER,TREVISO,home,...,Reginaldo,Luigi Beghetto,Pinga,Roberto Chiappara,Dino Fava,Jehad Muntasser,Adriano Zancope,Francesco Parravicini,Anderson,Alberto Giuliatto


In [2844]:
df.to_csv("train.csv")

### Data encoding
We need to encode the data before feeding it to the network. Here we define encoding methods that returns pytorch Tensors.

#### Seasons and Rounds

In [2845]:
class SeasonRoundEncoder(object):
    """Encode the season and round columns of the given pandas DataFrame sample"""
    def __init__(self, season_dict_map: dict):
        self.mapping = season_dict_map

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        season_encoding = torch.tensor([[el] for el in sample['season'].map(self.mapping).tolist()], dtype=torch.int32)
        round_encoding = torch.tensor([[el] for el in sample['round'].tolist()], dtype=torch.int32)
        return torch.cat([season_encoding, round_encoding], 1)

In [2846]:
season2index = {'20' + f'{i+5}'.zfill(2) + '-' + f'{i+6}'.zfill(2): i for i in range(16)}
season_round_encoder = SeasonRoundEncoder(season2index)

In [2847]:
# TEST seasons and rounds encoding
tensor = season_round_encoder(df.iloc[0:2])
seasons_rounds_expected_num_of_feats = 2
if tensor.shape[1] == seasons_rounds_expected_num_of_feats:
    print('SEASONS and ROUNDS encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {seasons_rounds_expected_num_of_feats}')
    raise Exception('SEASONS and ROUNDS encoding NOT OK! :(')

SEASONS and ROUNDS encoding OK


#### Datetime values

In [2848]:
class DatetimeEncoder(object):
    """Encode the year, month, day and hour columns of the given pandas DataFrame sample"""
    def __init__(self):
        pass

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        year_encoding = torch.tensor([[el] for el in sample['year'].tolist()], dtype=torch.int32)
        month_encoding = torch.tensor([[el] for el in sample['month'].tolist()], dtype=torch.int32)
        day_encoding = torch.tensor([[el] for el in sample['day'].tolist()], dtype=torch.int32)
        hour_encoding = torch.tensor([[el] for el in sample['hour'].tolist()], dtype=torch.int32)
        return torch.cat([year_encoding, month_encoding, day_encoding, hour_encoding], 1)

In [2849]:
datetime_encoder = DatetimeEncoder()

In [2850]:
# TEST datetime values encoding
tensor = datetime_encoder(df.iloc[0:2])
datetime_expected_num_of_feats = 4
if tensor.shape[1] == datetime_expected_num_of_feats:
    print('DATETIME encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {datetime_expected_num_of_feats}')
    raise Exception('DATETIME encoding NOT OK! :(')

DATETIME encoding OK


#### Results
One-hot encoding

In [2851]:
class ResultEncoder(object):
    """Encode the result column of the given pandas DataFrame sample"""
    def __init__(self, dict_map: dict):
        self.mapping = dict_map

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        return torch.tensor(sample['result'].map(self.mapping).tolist())

In [2852]:
result2onehot = {'home': [1, 0, 0], 'draw': [0, 1, 0], 'away': [0, 0, 1]}
result_encoder = ResultEncoder(result2onehot)

In [2853]:
# TEST results encoding
tensor = result_encoder(df.iloc[0:2])
results_expected_num_of_feats = len(df['result'].unique())
if tensor.shape[1] == results_expected_num_of_feats:
    print('RESULT encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {results_expected_num_of_feats}')
    raise Exception('RESULT encoding NOT OK! :(')

RESULT encoding OK


#### Referees
One-hot encoding

In [2854]:
class RefereeEncoder(object):
    """Encode the referee column of the given pandas DataFrame sample"""
    def __init__(self, lb: LabelBinarizer):
        self.lb = lb

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        return torch.tensor(self.lb.transform(sample['referee'].tolist()))

In [2855]:
lb = LabelBinarizer()
fitted_lb = lb.fit(df['referee'].tolist())
referee_encoder = RefereeEncoder(fitted_lb)

In [2856]:
# TEST referees encoding
tensor = referee_encoder(df.iloc[0:2])
referees_expected_num_of_feats = len(df['referee'].unique())
if tensor.shape[1] == referees_expected_num_of_feats:
    print('REFEREE encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {referees_expected_num_of_feats}')
    raise Exception('REFEREE encoding NOT OK! :(')

REFEREE encoding OK


#### Teams
One-hot encoding

In [2857]:
class TeamsEncoder(object):
    """Encode the home_team and away_team columns of the given pandas DataFrame sample"""
    def __init__(self, lb: LabelBinarizer):
        self.lb = lb

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        home_encoding = torch.tensor(self.lb.transform(sample['home_team'].tolist()))
        away_encoding = torch.tensor(self.lb.transform(sample['away_team'].tolist()))
        return torch.cat([home_encoding, away_encoding], 1)

In [2858]:
lb = LabelBinarizer()
# every team has played as home team at least once
fitted_lb = lb.fit(df['home_team'].tolist())
teams_encoder = TeamsEncoder(fitted_lb)

In [2859]:
# TEST teams encoding
tensor = teams_encoder(df.iloc[0:2])
teams_expected_num_of_feats = len(df['home_team'].unique()) * 2
if tensor.shape[1] == teams_expected_num_of_feats:
    print('TEAMS encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {teams_expected_num_of_feats}')
    raise Exception('TEAMS encoding NOT OK! :(')

TEAMS encoding OK


#### Coaches
One-hot encoding

In [2860]:
class CoachesEncoder(object):
    """Encode the home_team_coach and away_team_coach columns of the given pandas DataFrame sample"""
    def __init__(self, lb: LabelBinarizer):
        self.lb = lb

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        home_coach_encoding = torch.tensor(self.lb.transform(sample['home_team_coach'].tolist()))
        away_coach_encoding = torch.tensor(self.lb.transform(sample['away_team_coach'].tolist()))
        return torch.cat([home_coach_encoding, away_coach_encoding], 1)

In [2861]:
lb = LabelBinarizer()
# every team has played as home team at least once, so home_team_coach already contains all the coaches
fitted_lb = lb.fit(df['home_team_coach'].tolist())
coaches_encoder = CoachesEncoder(fitted_lb)

In [2862]:
# TEST coaches encoding
tensor = coaches_encoder(df.iloc[0:2])
coaches_expected_num_of_feats = len(df['home_team_coach'].unique()) * 2
if tensor.shape[1] == coaches_expected_num_of_feats:
    print('COACH encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {coaches_expected_num_of_feats}')
    raise Exception('COACH encoding NOT OK! :(')

COACH encoding OK


#### Players
One-hot encoding. We treat all players equally, both those that are part of the lineup and the substitutes

In [2863]:
class PlayersEncoder(object):
    """Encode the home and away team lineup and substitute players of the given pandas DataFrame sample"""
    def __init__(self, lb: LabelBinarizer):
        self.lb = lb

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        result = []
        for i in range(1, 12):
            result += [torch.tensor(self.lb.transform(sample[f'home_player_{i}'].tolist()))]
        for i in range(1, 8):
            result += [torch.tensor(self.lb.transform(sample[f'home_substitute_{i}'].tolist()))]
        for i in range(1, 12):
            result += [torch.tensor(self.lb.transform(sample[f'away_player_{i}'].tolist()))]
        for i in range(1, 8):
            result += [torch.tensor(self.lb.transform(sample[f'away_substitute_{i}'].tolist()))]
        return torch.cat(result, 1)

In [2864]:
def flatten_list(list_of_lists: list[list[str]]) -> list[str]:
    return [item for sublist in list_of_lists for item in sublist]


def encode_fit_players(source_df: pd.DataFrame) -> LabelBinarizer:
    lb = LabelBinarizer()
    player_cols = get_column_names_containing_str(source_df, 'home_player')
    player_cols += get_column_names_containing_str(source_df, 'home_substitute')
    all_players_unflattened = source_df.loc[:, player_cols].values.tolist()
    all_players_flattened = flatten_list(all_players_unflattened)
    lb.fit(all_players_flattened)
    return lb

In [2865]:
lb = LabelBinarizer()
fitted_lb = encode_fit_players(df)
players_encoder = PlayersEncoder(fitted_lb)

In [2866]:
# TEST players encoding
player_cols = get_column_names_containing_str(df, 'home_player')
player_cols += get_column_names_containing_str(df, 'home_substitute')
tensor = players_encoder(df.iloc[0:2])
all_unique_player_names = pd.concat([df[player_cols[i]] for i in range(len(player_cols))], axis=0).unique()
players_expected_num_of_feats = len(all_unique_player_names) * (11 + 7) * 2
if tensor.shape[1] == players_expected_num_of_feats:
    print('PLAYER encoding OK')
else:
    print(f'num of features: {tensor.shape[1]}')
    print(f'expected num of features: {players_expected_num_of_feats}')
    raise Exception('PLAYER encoding NOT OK! :(')

PLAYER encoding OK


In [2867]:
class Encode(object):
    """Encode the given pandas DataFrame sample and return a pytorch Tensor"""
    def __init__(self, season_round_enc: SeasonRoundEncoder, datetime_enc: DatetimeEncoder,
                 result_enc: ResultEncoder, referee_enc: RefereeEncoder, teams_enc: TeamsEncoder,
                 coaches_enc: CoachesEncoder, players_enc: PlayersEncoder,
                 keep_scores: bool, keep_result: bool):
        self.season_round_encoder = season_round_enc
        self.datetime_encoder = datetime_enc
        self.result_encoder = result_enc
        self.referee_encoder = referee_enc
        self.teams_encoder = teams_enc
        self.coaches_encoder = coaches_enc
        self.players_encoder = players_enc
        self.keep_scores = keep_scores
        self.keep_result = keep_result

    def __call__(self, sample: pd.DataFrame) -> torch.tensor:
        encoded = torch.cat([
            self.season_round_encoder(sample),
            self.datetime_encoder(sample),
            self.referee_encoder(sample),
            self.teams_encoder(sample),
            self.coaches_encoder(sample),
            self.players_encoder(sample)
        ], 1)
        if self.keep_scores:
            encoded = self.add_encoded_scores(encoded, sample)
        if self.keep_result:
            return self.add_encoded_result(encoded, sample)
        return encoded

    def add_encoded_scores(self, target: torch.tensor, source: pd.DataFrame):
         return torch.cat([
                target,
                torch.tensor([[el] for el in source['home_team_score'].tolist()], dtype=torch.int32),
                torch.tensor([[el] for el in source['away_team_score'].tolist()], dtype=torch.int32)
            ], 1)

    def add_encoded_result(self, target: torch.tensor, source: pd.DataFrame):
        return torch.cat([target, self.result_encoder(source)], 1)

In [2868]:
full_encoder = Encode(season_round_encoder, datetime_encoder, result_encoder, referee_encoder, teams_encoder, coaches_encoder, players_encoder, True, True)

no_result_encoder = Encode(season_round_encoder, datetime_encoder, result_encoder, referee_encoder, teams_encoder, coaches_encoder, players_encoder, True, False)

basic_encoder = Encode(season_round_encoder, datetime_encoder, result_encoder, referee_encoder, teams_encoder, coaches_encoder, players_encoder, False, False)

In [2869]:
# print train tensor example
test_sample = df.iloc[0:2]
test_encoded_sample = full_encoder(test_sample)
total_num_of_features = seasons_rounds_expected_num_of_feats \
                        + datetime_expected_num_of_feats \
                        + results_expected_num_of_feats \
                        + referees_expected_num_of_feats \
                        + teams_expected_num_of_feats \
                        + coaches_expected_num_of_feats \
                        + players_expected_num_of_feats \
                        + 2
if test_encoded_sample.shape[1] == total_num_of_features:
    print("encoding OK")
else:
    print(f'num of features: {test_encoded_sample.shape[1]}')
    print(f'expected num of features: {total_num_of_features}')
    raise Exception("encoding NOT OK")



encoding OK


In [2870]:
print(f'Total number of encoded features: {test_encoded_sample.size(1)}')

Total number of encoded features: 18679


In [2871]:
del df
del lb

### Data normalization

In [2872]:
# todo

### Dataset construction

We need to define a torch Dataset and torch Dataloader that will be used during training.

In [2873]:
def fill_with_padding(source: pd.DataFrame):
    if len(source) < 5:
        initial_len = len(source)
        padding = source.tail(1)
        for i in range(5-len(source)):
            source = pd.concat([source, padding], ignore_index=True)
        print(f'padding applied. Initial len: {initial_len} new_len: {len(source)}')
    return source


class SerieAFootballMatchesDataset(Dataset):
    history_len = 5

    def __init__(self, csv_file, x_encoder: Encode, x_historical_encoder: Encode, y_encoder: Encode):
        self.dataframe = pd.read_csv(csv_file)
        self.x_encoder = x_encoder
        self.x_historical_encoder = x_historical_encoder
        self.y_encoder = y_encoder

    def __len__(self) -> int:
        return len(self.dataframe)

    def __getitem__(self, idx):
        print(f'picked index {idx}')
        idx = self.scale_idx(idx)
        x_as_df = self.dataframe.iloc[[idx]] # df
        x = self.dataframe.loc[idx] # series
        # encoder expects a dataframe with a 'result' column
        y = pd.DataFrame([x['result']], columns=['result'])
        last_n_games_home, last_n_games_away = self.retrieve_historical_data(x)
        fill_with_padding(last_n_games_home)
        fill_with_padding(last_n_games_away)
        x, x_historical_home, x_historical_away, y = self.encode(x_as_df, last_n_games_home, last_n_games_away, y)
        return x, x_historical_home, x_historical_away, y

    def scale_idx(self, idx: int) -> int:
        """Scale the given index to a range that allows for historical data retrieval"""
        old_min = 0
        old_max = len(self.dataframe)
        '''
        idx = 10 corresponds to the first match of the second round.
        This ensure the retrieval of at least 1 historical match.
        In the worst case scenario, padding will fill the other 4 historical slots.
        '''
        new_min = 10
        new_max = old_max
        old_range = old_max - old_min
        new_range = new_max - new_min
        normalized_idx = (idx - old_min) / old_range
        return int(round(normalized_idx * new_range + new_min))

    def retrieve_historical_data(self, source: pd.DataFrame):
        """Retrieve historical data for home and away teams from source"""
        last_n_games_home = get_last_n_matches_played_by_team_before_round(
            self.dataframe, source['home_team'], source['round'], self.history_len)
        last_n_games_away = get_last_n_matches_played_by_team_before_round(
            self.dataframe, source['away_team'], source['round'], self.history_len)
        return last_n_games_home, last_n_games_away

    def encode(self, x: pd.DataFrame, x_historical_home: pd.DataFrame, x_historical_away: pd.DataFrame, y: pd.DataFrame):
        """Encode the data"""
        x_historical_home_encoded = self.x_historical_encoder(x_historical_home)
        x_historical_away_encoded = self.x_historical_encoder(x_historical_away)
        x_encoded = self.x_encoder(x)
        y_encoded = self.y_encoder(y)
        return x_encoded, x_historical_home_encoded, x_historical_away_encoded, y_encoded


In [2874]:
# test
t = torch.tensor([[[0, 0],
               [0, 1],
               [0, 2],
               [0, 3],
               [0, 4]],
              [[1, 0],
               [1, 1],
               [1, 2],
               [1, 3],
               [1, 4]],
              [[2, 2],
               [2, 1],
               [2, 2],
               [2, 3],
               [2, 4]]])
torch.reshape(t[0,0], (1,2))

tensor([[0, 0]])

## Training

In [2875]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.linear = nn.Linear(input_size + hidden_size, hidden_size)
        self.tanh = nn.Tanh()

    def forward(self, input, hidden):
        combined = torch.cat([input, hidden], 1)
        pre_hidden = self.linear(combined)
        hidden = self.tanh(pre_hidden)
        return hidden

    def init_hidden(self, minibatch_size):
        return torch.zeros(minibatch_size, 1, self.hidden_size)


class NeuralNetwork(nn.Module):
    def __init__(self, input_size):
        super(NeuralNetwork, self).__init__()
        self.input_size = input_size
        self.flatten = nn.Flatten()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 3),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        # 'x' comes in as:                minibatch_size x 1 x num_of_feats
        # 'rnn_*_hidden' will be:         minibatch_size x 1 x num_of_feats
        # --> they both need to be flattened
        x = self.flatten(x)
        output = self.layers(x)
        return output


class HybridNetwork(nn.Module):
    def __init__(self, rnn_home_model: RNN, rnn_away_model: RNN, mlp_model: NeuralNetwork):
        super(HybridNetwork, self).__init__()
        self.rnn_home = rnn_home_model
        self.rnn_away = rnn_away_model
        self.mlp = mlp_model

    def forward(self, x, x_historical_home, x_historical_away):
        # 'x' comes in as:                minibatch_size x 1 x num_of_feats
        # 'x_historical_*' comes in as:   minibatch_size x 5 x num_of_feats
        # 'rnn_*_hidden' will be:         minibatch_size x 1 x num_of_feats
        batch_size = x.size(0)
        time_seq_len = x_historical_home.size(1)
        num_of_feats = x_historical_home.size(2)
        # RNN HOME FORWARD
        rnn_home_hidden = self.rnn_home.init_hidden(batch_size)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_home_hidden[batch_idx] = self.rnn_home(
                    torch.reshape(x_historical_home[batch_idx, history_idx], (1, num_of_feats)),
                    rnn_home_hidden[batch_idx])
        # RNN AWAY FORWARD
        rnn_away_hidden = self.rnn_away.init_hidden(batch_size)
        for batch_idx in range(batch_size):
            for history_idx in range(time_seq_len):
                rnn_away_hidden[batch_idx] = self.rnn_away(
                    torch.reshape(x_historical_away[batch_idx, history_idx], (1, num_of_feats)),
                    rnn_away_hidden[batch_idx])
        # MLP FORWARD
        # concat on the features dimension
        print(x.shape)
        print(rnn_home_hidden.shape)
        print(rnn_away_hidden.shape)
        x_train = torch.cat([x, rnn_home_hidden, rnn_away_hidden], dim=2)
        print(x_train.shape)
        y_hat = self.mlp(x_train)
        return y_hat

In [2876]:
def train_epoch(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, loss_meter):
    for x, x_historical_home, x_historical_away, y in dataloader:
        # print(x.shape)
        # print(f'home_size {x_historical_home.shape}')
        # print(f'away_size {x_historical_away.shape}')
        if x_historical_home.size(1) == 5 & x_historical_away.size(1) == 5:
            optimizer.zero_grad()
            y_hat = model(x, x_historical_home, x_historical_away)
            print(f'y: {y}')
            print(f'y_hat: {y_hat}')
            loss = loss_fn(y, y_hat)
            loss.backward()
            optimizer.step()
            loss_meter.update(val=loss.item(), n=x.shape[0])
        else:
            print(f'skipping...')

def train_model(model: HybridNetwork, dataloader: DataLoader, optimizer: optim.Optimizer, loss_fn, num_epochs: int):
    model.train()
    for epoch in range(num_epochs):
        loss_meter = AverageMeter()
        train_epoch(model=model, dataloader= dataloader, optimizer=optimizer, loss_fn=loss_fn, loss_meter=loss_meter)
        print(f"Epoch {epoch+1} completed. Training loss: {loss_meter.avg}")

In [2877]:
train_dataset = SerieAFootballMatchesDataset(
    csv_file='train.csv', x_encoder=basic_encoder, x_historical_encoder=full_encoder, y_encoder=result_encoder)

In [2878]:
train_dataloader = DataLoader(train_dataset, batch_size=3, shuffle=True)
# test_dataloader = DataLoader(test_data, batch_size=64, shuffle=True)

In [2879]:
learning_rate = 0.01
num_epochs = 3
hidden_size = 128
rnn_home = RNN(input_size=total_num_of_features, hidden_size=hidden_size)
rnn_away = RNN(input_size=total_num_of_features, hidden_size=hidden_size)
# we have two hidden states (for home and away team) plus all features except for 'home_score', 'away_score' and 'result'
mlp = NeuralNetwork(hidden_size * 2 + (total_num_of_features - 3))
model = HybridNetwork(rnn_home_model=rnn_home, rnn_away_model=rnn_away, mlp_model=mlp)
nll_loss_fn = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [2880]:
train_model(model=model, dataloader=train_dataloader, optimizer=optimizer, loss_fn=nll_loss_fn, num_epochs=num_epochs)

picked index 22
padding applied. Initial len: 3 new_len: 5
padding applied. Initial len: 3 new_len: 5
picked index 170
picked index 123


RuntimeError: stack expects each tensor to be equal size, but got [3, 18679] at entry 0 and [5, 18679] at entry 1

# Missing data
- We don't have data about new players that come to play in _Serie A_ during the course of the seasons. The model has to learn from zero context how important their contribution is for the outcome of the matches. If we were to considered multiple leagues, we could keep track of player transfers and maintain the history.
- We don't have data about cup matches played during the course of the seasons, like _Champions League_, _Europa League_ and _Coppa Italia_. Since they are very prestigious competitions and matches are usually very competitive, teams put a lot of effort in them and therefore can then perform worse in the championship.
- We don't have any type of player performance metric like who scored a goal, who was the assist man, red or yellow cards, goalkeeper's saves etc. so the model could face some difficulties in learning which player is important for the team.